In [89]:
import pandas as pd
import extruct
import requests
from time import sleep
from w3lib.html import get_base_url
from urllib.parse import urlparse
from bs4 import BeautifulSoup

In [76]:
test_dic = {'maserati': ['Ghibli', 'GranTurismo'], 'nissan': ['sentra', 'maxima']}

In [120]:
def url_maker(dic):
    url_list = []
    for key, values in dic.items():
        for value in values:
            for year in range(2012, 2022):
                url_list.append(f'https://www.carvana.com/cars/{key}-{value}/{year}')
    return url_list

In [134]:
#url_list = [url, url2]
#url_list = url_maker(test_dic)
url_list = ['https://www.carvana.com/cars/nissan-sentra/2018-2022']

In [135]:
# the beginning 10 lines in this fuction to extract metadata and some imports taken from:
# https://practicaldatascience.co.uk/data-science/how-to-scrape-schemaorg-metadata-using-python

def extract_metadata(url_list):
    """Extract all metadata present in the page and return a dictionary of metadata lists. 
    
    Args:
        url (string): URL of page from which to extract metadata. 
    
    Returns: 
        metadata (dict): Dictionary of json-ld, microdata, and opengraph lists. 
        Each of the lists present within the dictionary contains multiple dictionaries.
    """
    df = None
    for url in url_list:
        sleep(1)
        r = requests.get(url)
        html = r.text
        base_url = get_base_url(r.text, r.url)
        metadata = extruct.extract(r.text, 
                                   base_url=base_url,
                                   uniform=True,
                                   syntaxes=['json-ld',
                                             'microdata',
                                             'opengraph'])
        soup = BeautifulSoup(html)
        
        ulist = soup.find_all('div', {'class': 'trim-mileage'})
        trim_list = []
        for u in ulist:
            trim_list.append(u.span.text)
        
        dic = metadata
        print(len(trim_list))
        print(len(dic['json-ld']))
        # is this right?
        if len(trim_list) <= len(dic['json-ld']):
            if df is not None:
                temp = pd.DataFrame.from_dict(dic['json-ld'][:len(trim_list)])
                temp['trim'] = trim_list
                df = pd.concat([df, temp], ignore_index=True)
                print(url, len(trim_list))
            else:
                df = pd.DataFrame.from_dict(dic['json-ld'][:len(trim_list)])
                df['trim'] = trim_list
                print(url, len(trim_list))
    return df

In [136]:
df = extract_metadata(url_list)

0
0
https://www.carvana.com/cars/nissan-sentra/2018-2022 0


In [112]:
df['price'] = df['offers'].map(get_price)

In [113]:
def get_price(line):
    return int(line.get('price'))

In [114]:
df

,@context,@type,itemCondition,name,modelDate,manufacturer,model,color,image,brand,description,mileageFromOdometer,sku,vehicleIdentificationNumber,offers,trim,price
0,http://schema.org,Vehicle,Used,2017 Maserati Ghibli,2017,Maserati,Ghibli,White,//cdnblob.fastly.carvana.io/2001892287/post-la...,Maserati,Used 2017 Maserati Ghibli undefined with 26513...,26513,2001892287,ZAM57XSA0H1225472,"{'@type': 'Offer', 'price': '37990', 'priceCur...",base,37990
1,http://schema.org,Vehicle,Used,2014 Maserati Ghibli,2014,Maserati,Ghibli,Silver,//cdnblob.fastly.carvana.io/2001848886/post-la...,Maserati,Used 2014 Maserati Ghibli undefined with 33866...,33866,2001848886,ZAM57RTA8E1089646,"{'@type': 'Offer', 'price': '32990', 'priceCur...",S Q4,32990
2,http://schema.org,Vehicle,Used,2017 Maserati Ghibli,2017,Maserati,Ghibli,Black,//cdnblob.fastly.carvana.io/2001723338/post-la...,Maserati,Used 2017 Maserati Ghibli undefined with 24269...,24269,2001723338,ZAM57RSAXH1242419,"{'@type': 'Offer', 'price': '42990', 'priceCur...",S,42990
3,http://schema.org,Vehicle,Used,2014 Maserati Ghibli,2014,Maserati,Ghibli,Blue,//cdnblob.fastly.carvana.io/2001673759/post-la...,Maserati,Used 2014 Maserati Ghibli undefined with 42641...,42641,2001673759,ZAM57RTA0E1089639,"{'@type': 'Offer', 'price': '33590', 'priceCur...",S Q4,33590
4,http://schema.org,Vehicle,Used,2014 Maserati Ghibli,2014,Maserati,Ghibli,Other,//cdnblob.fastly.carvana.io/2001829133/post-la...,Maserati,Used 2014 Maserati Ghibli undefined with 29367...,29367,2001829133,ZAM57RTAXE1087669,"{'@type': 'Offer', 'price': '34590', 'priceCur...",S Q4,34590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,http://schema.org,Vehicle,Used,2014 Nissan Maxima,2014,Nissan,Maxima,Blue,//cdnblob.fastly.carvana.io/2001828310/post-la...,Nissan,Used 2014 Nissan Maxima undefined with 60636 m...,60636,2001828310,1N4AA5AP3EC430854,"{'@type': 'Offer', 'price': '17990', 'priceCur...",S,17990
69,http://schema.org,Vehicle,Used,2017 Nissan Maxima,2017,Nissan,Maxima,Red,//cdnblob.fastly.carvana.io/2001452835/post-la...,Nissan,Used 2017 Nissan Maxima undefined with 42415 m...,42415,2001452835,1N4AA6AP9HC397885,"{'@type': 'Offer', 'price': '26990', 'priceCur...",SL,26990
70,http://schema.org,Vehicle,Used,2017 Nissan Maxima,2017,Nissan,Maxima,Silver,//cdnblob.fastly.carvana.io/2001725273/post-la...,Nissan,Used 2017 Nissan Maxima undefined with 60256 m...,60256,2001725273,1N4AA6AP1HC384788,"{'@type': 'Offer', 'price': '25990', 'priceCur...",SR,25990
71,http://schema.org,Vehicle,Used,2018 Nissan Maxima,2018,Nissan,Maxima,White,//cdnblob.fastly.carvana.io/2001800902/post-la...,Nissan,Used 2018 Nissan Maxima undefined with 90102 m...,90102,2001800902,1N4AA6AP2JC367388,"{'@type': 'Offer', 'price': '23590', 'priceCur...",SV,23590
